Наверное, можно было бы использовать ete3 для решения этой задачи на практике, но полагаю, что от меня требуется захардкодить это добро

In [ ]:
def parse_newick(newick):
    ''' 
    парсит строку в заданном формате (Newick) и возвращает дерево в виде вложенных списков
    '''
    stack = []
    current = []
    for char in newick:
        if char == '(':
            stack.append(current)
            current = []
        elif char == ')':
            if stack:
                subtree = current
                current = stack.pop()
                current.append(subtree)
        elif char == ',':
            continue
        elif char == ';':
            continue
        else:
            current.append(char)
    return current

def get_leaf_set(tree):
    ''' 
    возвращает множество листьев дерева (надо для сравнения)
    '''
    if isinstance(tree, str):
        return {tree}
    leaf_set = set()

    for subtree in tree:
        leaf_set.update(get_leaf_set(subtree))
    return leaf_set

def are_trees_equivalent(tree1, tree2):
    ''' 
    проверка на эквивалентность
    '''
    if isinstance(tree1, str) and isinstance(tree2, str):
        return tree1 == tree2
    if isinstance(tree1, str) or isinstance(tree2, str):
        return False
    if len(tree1) != len(tree2):
        return False
    tree1_leaf_sets = sorted([get_leaf_set(subtree) for subtree in tree1], key=len)
    tree2_leaf_sets = sorted([get_leaf_set(subtree) for subtree in tree2], key=len)
    return tree1_leaf_sets == tree2_leaf_sets

def get_leaf_mapping(tree1, tree2, mapping=None):
    ''' 
    строим соответствие между листьями для эквивалентных деревьев
    '''
    if mapping is None:
        mapping = {}
    if isinstance(tree1, str) and isinstance(tree2, str):
        mapping[tree1] = tree2
        return mapping
    for subtree1, subtree2 in zip(tree1, tree2):
        get_leaf_mapping(subtree1, subtree2, mapping)
    return mapping

def main(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    print(lines)
    tree1 = parse_newick(lines[0].strip())
    tree2 = parse_newick(lines[1].strip())
    
    if len(get_leaf_set(tree1)) < 6 or len(get_leaf_set(tree2)) < 6:    # проверяем количество листьев
        print(None)
        return
    
    if are_trees_equivalent(tree1, tree2):
        mapping = get_leaf_mapping(tree1, tree2)
        print(list(mapping.items()))
    else:
        print(None)


['(X,(Y,Z));    \n', '(A,(B,C)); \n', '\n']
None


# Tests

In [32]:
file_path = "test_1.txt"  
main(file_path)

['"(A,(B,(C,D)));"\n', '"(C,(A,(D,B)));"']
None


In [33]:
file_path = "test_2.txt"  
main(file_path)

['((A,B),(C,(D,(E,F)))); \n', '((C,(D,(F,E))),(B,A)); \n', '\n']
[('A', 'C'), ('B', 'D'), ('C', 'B'), ('D', 'A')]


In [34]:
file_path = "test_3.txt"  
main(file_path)

['(X,(Y,Z));    \n', '(A,(B,C)); \n', '\n']
None


In [35]:
file_path = "test_4.txt"  
main(file_path)

['(A,(B,(C,D)));  \n', '((C,D),B,A);\n']
None
